<h2>Aquele 1% é probabilidade </h2>

Nesse tutorial vamos ver uma maneira bem simples de gerar textos automaticamente, pra isso utilizaremos probabilidade condicional e como fonte de dados músicas do Wesley Safadão :p

Essa é uma técnica bem simples, baseada na propriedade de Markov, que diz que em um processo aleatório de tempo discreto, a distribuição de probabilidade condicional para o próximo passo ($t+1$) depende apenas do estado atual, e não dos momentos anteriores. (Sabemos que essa prorpiedade não é 100% válida para textos, já que existe uma grande dependência entre os assuntos, mas ela pode nos retornar resultados interessantes).

Além do uso de probabilidade condicional para gerar textos, também vamos brincar um pouco com a estrutura de dados do python, usando dicionários para acessar de forma eficiente a distribuiçao de probabilidade das lestras de Wesley. Também temos uma breve introdução da biblioteca lxml.html, o qual otilizamos para parsear as letras de música do letras.com

In [1]:
import lxml.html as parser
import requests
import urllib
import json
import re
import numpy as np

from itertools import chain
from collections import defaultdict
from os import path

Vamos montar uma estrutura (json) para armazenarmos as os nomes e letras de cada música, que será nossa base de dados.
O código pode parecer um pouco confuso, porém é simples. </br>

De iníco vamos fazer um request na nossa url-base, onde estao listadas todas as músicas do autor que estamos procurando </br>
Dentro da classe cnt-list, no html da página base, estão listados todos os links para as letras do autos, desta forma vamos gerar uma lista com todos esse links para podermos navegar por eles. </br>
Por último, chamos a função get_song(), a qual enviamos o link de cada música e e obtemos a letra que está no div cnt-letra p402_premium. </br>

Nosso dicionário songs tera a forma {nome_musica: lista de versos da musica}, como pode ser observado abaixo.


In [2]:
def load_songs(autor):
    def get_song(url):
        r = requests.get(url) 
        song_html = parser.fromstring(r.text)
        song = song_html.xpath("//div[@class='cnt-letra p402_premium']/article/p/text()")
        return song
    
    filename = autor+'.json'
    if path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    else:
        base_url = urllib.parse.urljoin('https://www.letras.mus.br/', autor)
        r = requests.get(start_url)
        html = parser.fromstring(r.text)
        links = html.xpath("//ul[@class='cnt-list']/li/a/@href")
        links = [urllib.parse.urljoin(base_url,l) for l in links]

        songs = {song_url.split('/')[-2]: get_song(song_url) 
                 for song_url in links}
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(songs, file, ensure_ascii=False)
        
    return songs

In [37]:
songs_dict = load_songs('wesley-safadao')

In [40]:
songs_dict['100-amor'][:3]

['Assim é o nosso amor', 'io io io, io io iooo', '100% amor']

In [42]:
songs_dict['100-muito-louco'][:3]

['10% de Red Bull', '10% de água de coco', '80% de uísque']

Criamos uma função para limpar levemente os dados, removendo apenas pontuações e textos entre parênteses (bis), (3X). Também adicionamos dois marcadores em cada verso, o > que marca início de uma sentença e <, que marca o fim.
Desta forma, esses caracteres no auxiliaram a identificar quando uma frase deve ser completa, pois calcularemos a probabilidade de a frase terminar, dado que obtemos a palavra $w$, $P(<|w)$ e também forcecerá informação para sabermos se a frase está iniciando, ou seja, qual a probabilidade de uma palavra ocorrer, dado que é início da frase $P(>|w)$.

In [3]:
def process_song(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = '> ' + text + ' <'
    return text.split()

Como para nós o que importa são os versos, e não a letra toda, vamos agrupar todas as músicas em uma lista de versos. Vamos utilizar o comando chain, da itertools para desacoplarmos as sublistas em apenas uma lista. Após isso, dividimos os versos em tokens (palavras), pois precisaremos contar a ocorrência de cada uma no texto.

In [5]:
corpora = list(chain(*songs_dict.values()))
tokenized_corpora = [process_song(verse) for verse in corpora]

In [45]:
tokenized_corpora[:3]

[['>', 'assim', 'é', 'o', 'nosso', 'amor', '<'],
 ['>', 'io', 'io', 'io', 'io', 'io', 'iooo', '<'],
 ['>', '100', 'amor', '<']]

Para calcularmos a probabilidade de um item A, dado que ocorreu B, precisamos utilizar a regra de Bayes $P(A|B) = \frac{P(A\cap B)}{P(B)}$ </br>
Para isso, vamos criar dois dicionários do tipo defauldict para armazenar as ocorrêncas $(A)$ e coocorrências $A\cap B)$ dos tokens no texto. O defaultdict nos ajuda nessa tarefa, uma vez que solicitada uma chave não presente no nosso dicionário, ele retorna um valor default, 0 no nosso caso.

In [46]:
cooccurencies = defaultdict(lambda: 0)
occurencies = defaultdict(lambda: 0)

Varremos todo nosso corpora com uma janela deslizante de tamanho dois, ou seja, para o verso ['>', 'assim', 'é', 'o', 'nosso', 'amor', '<'] teremos os tokens (>, assim), (assim, é), (é, o), .... que é a ocorrência do segundo item, dado que o primeiro apareceu.
Vamos contar todas essas coocorrências e a ocorrência de cada token individalmente, uma vez que precisamos desses valores para calcular nossa probabilidade com a equação de Bayes.

In [47]:
for verse in tokenized_corpora:
    for i in range(len(verse)-1):
        cooccurencies[(verse[i], verse[i+1])] += 1
        occurencies[verse[i]] += 1
occurencies['<'] = occurencies['>']

In [60]:
cooccurencies[('assim', 'é')]

9

In [59]:
occurencies['assim']

287

vamos salvar todas as nossas palavras do vocabulário em uma lista, pois precisaremos montar uma função de probabilidade para cada palavra do vocabulário e acessá-la depois

In [11]:
vocabulary = list(occurencies.keys())

Agora, vamos montar uma outra estrutura para armazenar a probabilidade conjunta de cada palavra $w_i$ ocorrer, dada a palavra $w_{i-1}$. Será um dicionário de dicionários, onde a chave mais externa corresponde à palavra dada e a mais interna à probabilidade de ela ocorrer condicionada à anterior.

In [61]:
condition_probs = defaultdict(lambda: defaultdict(lambda: 0))
for condition in cooccurencies.keys():
    condition_probs[condition[0]][condition[1]] = cooccurencies[condition]/occurencies[condition[0]]

In [ ]:
#condition_probs

In [64]:
condition_probs['>']['assim']

0.000947205743057979

e a probabilidade de 'é', dado que 'assim' ocorreu:

In [65]:
condition_probs['assim']['é']

0.0313588850174216

Agora está muito simples gerarmos nossos textos, uma vez que temos a função de distribuição de probabilidade de todos os pares de palavra, precisamos apenas sorteá-las de acordo com a a probabilidade de cada uma ocorrer, o que alterado a cada novo sorteio, já que a palavra $w_i$ fornece informação sobre a ocorrência da palavra $w_{i+1}$.

In [71]:
for i in range(10):
    fact = '>'
    verse = []
    while fact != '<':
        probs = [condition_probs[fact][token] for token in vocabulary]
        fact = np.random.choice(vocabulary, 1, p=probs)[0]
        verse.append(fact)
    print(' '.join(verse))

no forró que tá feliz daqui pra lá chega em poucas palavras <
o seu travesseiro <
seus lábios de ser assim me vingar vingar vingar vingar <
que fazes <
tão difícil pra ganhar teu coração <
nem muito por ti esquecer <
e me quer cantar alegremente <
minha mente é tudo bem eu sou apaixonada <
vou me ver quem não eu sou o que é lindo dono do seu corpo fica ai <
o desmantelo <


probs é nossa lista de probabilidade de ocorrência para cada palavra do vocabulário, note que ela é refeita a cada palavra (fact) nova que sorteamos, pois isso alterá a probabilidade das próximas palavras $P(w_{i+1}|w_i)$. Essas probabilidade condicionais estão todas amrazenadas no nosso dicionário condition_probs.

A função np.random.choice sorteia um item da lista vocabulary, de acordo com a probabilidade de cada uma, definida no vetor probs.
Poderíamos sempre selecionar a palavra mais provável, porém perderíamos a dinâmica e estocacidade do modelo.

Apesar de ser um exemplo bem simples e não nos retornar nenhum resultado digno de um gremmy, é possível observar que as frases fazem mais sentido do que palavras aleatórias.

Esta técnica pode ser melhorada facilmente utilizando-se bigramas, ou gerando dois versos, condicionando a última palavra do segundo verso à última do primeiro (para mantermos a rima)